## Nanograd Demo

#### Imports & Config

In [65]:
from milligrad import Tensor
from milligrad.nn import Linear, MLP

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes

%matplotlib inline

#### Data

In [66]:
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target
y_mean, y_std = y.mean(), y.std()
y = (y - y_mean) / y_std  # normalize targets

#### Model